Role: Expert Python Developer & Quant Trader Task:  Instructions: Please update the following three files (src/kalshi_feed.py, src/market_scanner.py, and streamlit_app.py) with the code provided below and any additional code needed to implement the Hybrid Scanner. 

Changes: Implements cursor pagination to dig deeper than the first page of results. It also adds a robust Category cleaner to normalize API data.

In [ ]:
import requests
import os
import time
from dotenv import load_dotenv
from pathlib import Path

# Load environment variables
root_dir = Path(__file__).parent.parent
env_path = root_dir / '.env'
load_dotenv(dotenv_path=env_path, override=True)

KALSHI_API_URL = "https://api.elections.kalshi.com/trade-api/v2/markets"
API_KEY = os.getenv("KALSHI_API_KEY")

def get_all_active_markets(limit_pages=10):
    """
    Fetches active markets using pagination to bypass the 'Sports Wall'.
    limit_pages: Number of 1000-market pages to fetch (default 10 = 10k markets).
    """
    headers = {}
    if API_KEY:
        headers["Authorization"] = f"Bearer {API_KEY}"

    print(f"🔍 Deep Scan: Fetching up to {limit_pages * 1000} markets...")
    
    all_raw_markets = []
    cursor = None
    
    for page in range(limit_pages):
        params = {
            "limit": 1000,
            "status": "open",
            "with_nested_markets": True
        }
        if cursor:
            params["cursor"] = cursor

        try:
            response = requests.get(
                KALSHI_API_URL, 
                params=params, 
                headers=headers, 
                timeout=10
            )
            
            if response.status_code == 200:
                data = response.json()
                batch = data.get('markets', [])
                cursor = data.get('cursor')
                
                if not batch:
                    break # No more data
                
                all_raw_markets.extend(batch)
                print(f"   ✅ Page {page+1}: Fetched {len(batch)} markets. (Total: {len(all_raw_markets)})")
                
                if not cursor:
                    break # End of list
                
                time.sleep(0.2) # Rate limit politeness
            else:
                print(f"   ❌ API Error on page {page+1}: {response.status_code}")
                break
                
        except Exception as e:
            print(f"   ❌ Connection Error: {e}")
            break
            
    print(f"🏁 Fetch complete. Processing {len(all_raw_markets)} markets...")
    return clean_market_data(all_raw_markets)

def clean_market_data(raw_markets):
    """
    Categorizes and cleans market data.
    """
    cleaned = []
    
    for m in raw_markets:
        # 1. Liquidity Check
        yes_ask = m.get('yes_ask', 0)
        if yes_ask == 0: continue 

        # 2. Extract & Normalize Category
        # Kalshi categories are often in 'category', but sometimes implied by 'series_ticker'
        raw_cat = m.get('category', 'Uncategorized')
        title = m.get('title', '').lower()
        
        # Smart Categorization
        if raw_cat == 'Climate and Weather' or 'temperature' in title or 'rain' in title:
            category = 'Weather'
        elif raw_cat in ['Economics', 'Fed', 'Interest Rates']:
            category = 'Economics'
        elif raw_cat in ['Politics', 'Government']:
            category = 'Politics'
        elif raw_cat in ['Financials', 'Crypto', 'Stocks']:
            category = 'Financials'
        elif raw_cat in ['Sports', 'MLB', 'NBA', 'NFL', 'Soccer']:
            category = 'Sports'
        else:
            category = raw_cat

        # 3. Data Structure
        cleaned.append({
            'ticker': m.get('ticker'),
            'title': m.get('title', 'Unknown'),
            'category': category,
            'price': yes_ask,
            'no_price': m.get('no_ask', 0),
            'volume': m.get('volume', 0),
            'expiration': m.get('expiration_time'),
            'spread': abs(yes_ask - m.get('yes_bid', 0)),
            'raw': m # Keep raw for ML matching
        })
        
    # Sort by Volume for quality
    cleaned.sort(key=lambda x: x['volume'], reverse=True)
    return cleaned

Changes: * Added _scan_financial_ml which calls your original ML models (SPX, BTC, etc.).

Added _scan_weather and _scan_smart_money (Econ/Politics).

Filters out "Sports" from the general view (but keeps them for Arbitrage if the math works).

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timezone
from src.kalshi_feed import get_all_active_markets

# Import your existing ML pipeline
try:
    from src.data_loader import fetch_data
    from src.feature_engineering import create_features
    from src.model import load_model, predict_next_hour
except ImportError:
    print("⚠️ ML Modules not found. 'AI Predictor' will be disabled.")

class HybridScanner:
    def __init__(self):
        self.markets = []
        self.last_scan = None

    def run_scan(self):
        """
        Executes the Hybrid Scan.
        """
        self.markets = get_all_active_markets(limit_pages=10) # Deep scan
        self.last_scan = datetime.now(timezone.utc)
        
        return {
            "ml_alpha": self._scan_financial_ml(),
            "smart_money": self._scan_smart_money(),
            "weather": self._scan_weather(),
            "arbitrage": self._find_arbitrage(),
            "yield_farming": self._find_yield_farms()
        }

    def _scan_financial_ml(self):
        """
        Restores the functionality of your original SP500 Predictor.
        Scans SPX, BTC, ETH, Nasdaq using your trained LightGBM models.
        """
        signals = []
        targets = ["SPX", "BTC", "ETH", "Nasdaq"]
        
        # Filter markets to only Financials/Crypto to save time
        fin_markets = [m for m in self.markets if m['category'] in ['Financials', 'Crypto', 'Economics']]

        for ticker in targets:
            try:
                # 1. Fetch Data & Model
                df = fetch_data(ticker, period="5d", interval="1m")
                model, _ = load_model(ticker)
                
                if model and not df.empty:
                    # 2. Predict
                    df_feat = create_features(df)
                    pred_val = predict_next_hour(model, df_feat, ticker)
                    curr_price = df['Close'].iloc[-1]
                    
                    # 3. Match with Kalshi Markets
                    for m in fin_markets:
                        # Match Ticker logic (e.g. "SPX" in "INX")
                        is_match = False
                        if ticker == "SPX" and "INX" in m['ticker']: is_match = True
                        elif ticker == "Nasdaq" and "NAS" in m['ticker']: is_match = True
                        elif ticker in m['ticker'] or ticker in m['title']: is_match = True
                        
                        if not is_match: continue

                        # Parse Strike
                        try:
                            # Heuristic: Extract number from title
                            import re
                            strike_match = re.search(r'(\d{1,3}(?:,\d{3})*(?:\.\d+)?)', m['title'])
                            if not strike_match: continue
                            strike = float(strike_match.group(1).replace(',', ''))
                        except:
                            continue

                        # Determine Edge
                        direction = "Bullish" if (">" in m['title'] or "Above" in m['title']) else "Bearish"
                        
                        # Logic: If Model > Strike AND Market is "Above" -> BUY
                        if direction == "Bullish" and pred_val > strike:
                            conf = (pred_val - strike) / strike * 100 # Rough conf proxy
                            signals.append({
                                "Asset": ticker,
                                "Market": m['title'],
                                "Model_Pred": pred_val,
                                "Strike": strike,
                                "Kalshi_Price": m['price'],
                                "Action": "BUY YES",
                                "Edge": "Bullish",
                                "Confidence": f"{conf:.2f}%"
                            })
                        elif direction == "Bearish" and pred_val < strike:
                            signals.append({
                                "Asset": ticker,
                                "Market": m['title'],
                                "Model_Pred": pred_val,
                                "Strike": strike,
                                "Kalshi_Price": m['price'],
                                "Action": "BUY YES",
                                "Edge": "Bearish",
                                "Confidence": "High"
                            })
                            
            except Exception as e:
                print(f"Skipping ML for {ticker}: {e}")
                continue
        
        return signals

    def _scan_smart_money(self):
        """Filters for Economics and Politics (The 'Real' Markets)."""
        candidates = []
        for m in self.markets:
            if m['category'] in ['Economics', 'Politics']:
                candidates.append(m)
        return candidates

    def _scan_weather(self):
        """Filters for Weather/Climate."""
        candidates = []
        for m in self.markets:
            if m['category'] == 'Weather':
                candidates.append(m)
        return candidates

    def _find_arbitrage(self):
        """Finds negative spreads (Free Money) across ALL categories."""
        opps = []
        for m in self.markets:
            if m['no_price'] > 0:
                cost = m['price'] + m['no_price']
                if cost < 100:
                    opps.append({
                        **m,
                        "cost": cost,
                        "profit": 100 - cost
                    })
        return opps

    def _find_yield_farms(self):
        """Finds safe 92-98c bets (<48h) excluding Sports."""
        opps = []
        now = datetime.now(timezone.utc)
        for m in self.markets:
            if m['category'] == 'Sports': continue # Skip sports for safe yield
            
            if 92 <= m['price'] <= 98:
                if not m['expiration']: continue
                try:
                    exp = pd.to_datetime(m['expiration'])
                    if exp.tzinfo is None: exp = exp.replace(tzinfo=timezone.utc)
                    hours = (exp - now).total_seconds() / 3600
                    
                    if 0 < hours < 48:
                        roi = (100 - m['price']) / m['price'] * 100
                        opps.append({**m, "hours_left": int(hours), "roi": roi})
                except: continue
        
        opps.sort(key=lambda x: x['roi'], reverse=True)
        return opps

Changes: * Tab 1 (📈 AI Predictor): Restores your SPX/BTC model interface.

Tab 2 (💸 Sniper): The Yield/Arb strategies.

Tab 3 (⛈️ Weather): Dedicated Weather interface.

Tab 4 (🏛️ Macro): Economics & Politics (excluding Sports).

In [ ]:
import streamlit as st
import pandas as pd
from src.market_scanner import HybridScanner

st.set_page_config(page_title="Kalshi Pro Scanner", layout="wide", page_icon="⚡")

# --- CUSTOM CSS ---
st.markdown("""
<style>
    .metric-box {
        background-color: #1a1c24;
        border: 1px solid #333;
        padding: 15px;
        border-radius: 10px;
        margin-bottom: 10px;
    }
    .highlight-green { color: #00ff94; font-weight: bold; }
</style>
""", unsafe_allow_html=True)

# --- INIT ---
if 'scanner' not in st.session_state:
    st.session_state.scanner = HybridScanner()
if 'data' not in st.session_state:
    st.session_state.data = None

# --- SIDEBAR ---
with st.sidebar:
    st.title("⚡ Kalshi Pro")
    st.caption("Deep Scan (10k Markets) Active")
    
    if st.button("🔄 RUN DEEP SCAN", type="primary", use_container_width=True):
        with st.spinner("Fetching 10,000 markets to bypass sports..."):
            st.session_state.data = st.session_state.scanner.run_scan()
            st.rerun()
            
    st.markdown("---")
    st.info("Scanner filters out Sports by default to find 'Money Markets' (Econ, Politics, Weather).")

# --- MAIN ---
st.title("Market Opportunities")

if st.session_state.data:
    data = st.session_state.data
    
    # Overview Metrics
    c1, c2, c3, c4 = st.columns(4)
    c1.metric("Markets Scanned", len(st.session_state.scanner.markets))
    c2.metric("AI Signals", len(data['ml_alpha']))
    c3.metric("Weather Mkts", len(data['weather']))
    c4.metric("Arb Opps", len(data['arbitrage']))
    
    st.markdown("---")
    
    # TABS
    tab_ml, tab_sniper, tab_weather, tab_macro = st.tabs([
        "📈 AI Predictor", 
        "💸 Sniper (Easy Money)", 
        "⛈️ Weather", 
        "🏛️ Politics & Econ"
    ])
    
    # === TAB 1: AI PREDICTOR (Restored) ===
    with tab_ml:
        st.subheader("🤖 Machine Learning Signals")
        st.caption("Predictions based on your trained LightGBM models for SPX, Nasdaq, BTC, ETH.")
        
        if data['ml_alpha']:
            for sig in data['ml_alpha']:
                with st.container(border=True):
                    c1, c2, c3 = st.columns([3, 1, 1])
                    with c1:
                        st.markdown(f"**{sig['Market']}**")
                        st.caption(f"Asset: {sig['Asset']} | Pred: {sig['Model_Pred']:.2f}")
                    with c2:
                        st.markdown(f"Action: :green[**{sig['Action']}**]")
                        st.caption(f"Kalshi Price: {sig['Kalshi_Price']}¢")
                    with c3:
                        st.metric("Model Edge", sig['Edge'])
        else:
            st.info("No strong ML signals found matching current Kalshi markets.")

    # === TAB 2: SNIPER ===
    with tab_sniper:
        # Arbitrage Section
        if data['arbitrage']:
            st.error(f"🚨 {len(data['arbitrage'])} ARBITRAGE OPPORTUNITIES")
            for arb in data['arbitrage']:
                with st.container(border=True):
                    st.markdown(f"**{arb['title']}**")
                    st.markdown(f"Buy YES ({arb['price']}¢) + Buy NO ({arb['no_price']}¢) = **{arb['cost']}¢ Cost**")
                    st.markdown(f"Profit: :green[**+{arb['profit']}¢ per share**]")
        
        # Yield Farming Section
        st.subheader("💰 Yield Farming (Non-Sports)")
        st.caption("High probability (>92%) trades expiring in <48h. Economics, Politics, etc.")
        
        if data['yield_farming']:
            for farm in data['yield_farming']:
                with st.container(border=True):
                    c1, c2, c3 = st.columns([3, 1, 1])
                    with c1:
                        st.markdown(f"**{farm['title']}**")
                        st.caption(f"{farm['category']} • Vol: {farm['volume']:,}")
                    with c2:
                        st.metric("Price", f"{farm['price']}¢")
                    with c3:
                        st.metric("ROI", f"+{farm['roi']:.1f}%")
                        st.caption(f"{farm['hours_left']}h left")
        else:
            st.warning("No yield farms found.")

    # === TAB 3: WEATHER ===
    with tab_weather:
        st.subheader("⛈️ Climate & Weather")
        st.caption("Hurricanes, Temperature, Rain, Snow.")
        
        if data['weather']:
            # Search/Filter
            w_search = st.text_input("Filter Weather (e.g. 'NYC', 'Rain')", "")
            
            for w in data['weather']:
                if w_search.lower() in w['title'].lower():
                    with st.container(border=True):
                        c1, c2 = st.columns([3, 1])
                        with c1:
                            st.markdown(f"**{w['title']}**")
                            st.caption(f"Expires: {w['expiration']}")
                        with c2:
                            st.metric("Price", f"{w['price']}¢")
        else:
            st.info("No active weather markets found.")

    # === TAB 4: MACRO (ECON/POLITICS) ===
    with tab_macro:
        st.subheader("🏛️ Smart Money (Economics & Politics)")
        st.caption("Fed Rates, CPI, GDP, Elections.")
        
        if data['smart_money']:
            for m in data['smart_money']:
                with st.container(border=True):
                    c1, c2, c3 = st.columns([3, 1, 1])
                    with c1:
                        st.markdown(f"**{m['title']}**")
                        st.caption(m['category'])
                    with c2:
                        st.progress(m['price']/100, text=f"Prob: {m['price']}%")
                    with c3:
                        st.metric("Volume", f"{m['volume']:,}")
        else:
            st.info("No active Economics or Politics markets found.")

else:
    st.markdown("### 👋 Ready to Deep Scan")
    st.write("Click **RUN DEEP SCAN** in the sidebar to fetch 10k markets and filter out the noise.")